In [1]:
 %load_ext sql
import pandas as pd

In [2]:
# Load the cleaned CSV
df = pd.read_csv(r'C:\Users\Olamide\Downloads\cleaned_air_pollution_lagos.csv')
df.head(5)

,Date,Location,PM2.5 (µg/m³),CO (ppm),Temperature (°C),Wind Speed (km/h)
0,2025-05-01,IKEJA,44.6,1.68,34.5,5.3
1,2025-05-01,LEKKI,67.8,0.64,34.6,9.6
2,2025-05-01,SURULERE,44.2,1.68,30.6,17.0
3,2025-05-01,YABA,116.8,2.08,28.0,13.1
4,2025-05-01,APAPA,80.0,1.06,28.8,5.0


In [3]:
# Import the function needed to connect Python to an SQLite database
from sqlalchemy import create_engine

# Create a connection to a SQLite database file named 'air_pollution.sqlite'
file_engine = create_engine('sqlite:///air_pollution.sqlite')

In [4]:
# Save df as a table named 'air_pollution' in the SQLite database file
df.to_sql('air_pollution', con=file_engine, index=False, if_exists='replace')

59

In [5]:
# Connect to the 'air_pollution.sqlite' database 
%sql sqlite:///air_pollution.sqlite

In [6]:
%%sql
SELECT *
FROM air_pollution
LIMIT 5;

 * sqlite:///air_pollution.sqlite
Done.


Date,Location,PM2.5 (µg/m³),CO (ppm),Temperature (°C),Wind Speed (km/h)
2025-05-01,IKEJA,44.6,1.68,34.5,5.3
2025-05-01,LEKKI,67.8,0.64,34.6,9.6
2025-05-01,SURULERE,44.2,1.68,30.6,17.0
2025-05-01,YABA,116.8,2.08,28.0,13.1
2025-05-01,APAPA,80.0,1.06,28.8,5.0


# 1. Which location had the highest average PM2.5 level?

### PM2.5 is a fine particulate matter in the air, small enough to enter lungs and blood. High PM2.5 = more air pollution.

I calculated the average PM2.5 level for each location, rounded to 2 decimal places. Then sort them in descending order to find the most polluted areas

In [10]:
%%sql
SELECT 
    Location, 
    ROUND(AVG([PM2.5 (µg/m³)]), 2) AS avg_pm25
FROM air_pollution
GROUP BY Location
ORDER BY avg_pm25 DESC;

 * sqlite:///air_pollution.sqlite
Done.


Location,avg_pm25
YABA,83.38
IKEJA,74.96
SURULERE,67.85
LEKKI,66.1
AJAH,62.91
APAPA,62.5


YABA has the highest average PM2.5 level

# 2. Which locations consistently stay below the WHO PM2.5 limit (15 µg/m³)?

### According to the World Health Organization (WHO), PM2.5 levels above 15 µg/m³ are considered unsafe.

I Grouped the data by location and return only those where the highest PM2.5 reading is less than 15 µg/m³.


In [15]:
%%sql
SELECT
    Location, 
    [PM2.5 (µg/m³)]
FROM air_pollution
GROUP BY Location
HAVING MAX([PM2.5 (µg/m³)]) < 15;

 * sqlite:///air_pollution.sqlite
Done.


Location,PM2.5 (µg/m³)


None of the locations in this Lagos dataset consistently stayed within WHO safe limits 

# 3. What’s the average CO level across all locations?

### Carbon monoxide (CO) is a gas mostly from cars and generators. According to WHO and EPA: < 9 ppm is safe

I Calculated the average CO level (in ppm) across all locations, rounded to 2d.p

In [20]:
%%sql
SELECT 
    ROUND(AVG([CO (ppm)]), 2) AS average_co
FROM air_pollution;

 * sqlite:///air_pollution.sqlite
Done.


average_co
1.52


A CO level of 1.52 ppm is generally considered low and safe.

# 4. Do higher temperatures correlate with higher PM2.5 levels?

### Meaning: When the temperature increases, do PM2.5 levels also go up?

I Grouped temperatures into 3 categories (Cool, Warm, Hot). Then calculated the average PM2.5 level in each group to look for a correlation 


In [25]:
%%sql
WITH temp_groups AS (
    SELECT
        CASE
            WHEN [Temperature (°C)] < 27 THEN 'Cool (<27°C)'
            WHEN [Temperature (°C)] BETWEEN 27 AND 30 THEN 'Warm (27–30°C)'
            ELSE 'Hot (>30°C)'
        END AS temp_category,
        [PM2.5 (µg/m³)]
    FROM air_pollution
)

SELECT 
    temp_category,
    COUNT(*) AS readings_count,
    ROUND(AVG([PM2.5 (µg/m³)]), 2) AS avg_pm25
FROM temp_groups
GROUP BY temp_category
ORDER BY avg_pm25 DESC;

 * sqlite:///air_pollution.sqlite
Done.


temp_category,readings_count,avg_pm25
Warm (27–30°C),16,80.55
Hot (>30°C),43,65.7


The warm category has higher PM2.5 levels than the hot category. So, in this dataset, PM2.5 levels do not correlate with temperature.


# 5. Rank all locations by average pollution level

### Find the average of PM2.5 and CO for each location, then rank them.

I added the averages of PM2.5 and CO and found the average to compare pollution levels across locations. While the units are different and this isn't a scientifically accurate combination, it works for general ranking.

In [30]:
%%sql
WITH avg_pollution AS (
    SELECT 
        Location,
        ROUND(AVG([PM2.5 (µg/m³)]), 2) AS avg_pm25,
        ROUND(AVG([CO (ppm)]), 2) AS avg_co,
        ROUND((AVG([PM2.5 (µg/m³)]) + AVG([CO (ppm)])) / 2, 2) AS avg_pollution_score
    FROM air_pollution
    GROUP BY Location
),
ranked_pollution AS (
    SELECT *,
        DENSE_RANK() OVER (ORDER BY avg_pollution_score DESC) AS pollution_rank
    FROM avg_pollution
)

SELECT *
FROM ranked_pollution;

 * sqlite:///air_pollution.sqlite
Done.


Location,avg_pm25,avg_co,avg_pollution_score,pollution_rank
YABA,83.38,1.66,42.52,1
IKEJA,74.96,1.5,38.23,2
SURULERE,67.85,1.61,34.73,3
LEKKI,66.1,1.34,33.72,4
AJAH,62.91,1.37,32.14,5
APAPA,62.5,1.62,32.06,6
